Tasks:


1. We input CGS' daily returns data from Interactive Brokers


2. We clean the data to remove weekends


3. We convert daily data to monthly


4. We input CGS' official net monthly returns data from the administrator


5. We calculate/estimate daily returns for CGS based on the official monthly data from the administrator by using the daily data from Interactive Brokers as the starting point



In [95]:
import pandas as pd
import numpy as np
import math

In [96]:
data=pd.read_csv(r'C:\Users\Vedant Wanchoo\Desktop\CGS 2020\Data Requests\CGS_IB_Daily_2024-03-19.csv')

In [97]:
data.shape

(1978, 2)

In [98]:
data.tail()

,Date,CGS_IB_Daily
1973,25-02-2024,0.0002
1974,26-02-2024,0.0014
1975,27-02-2024,-0.0082
1976,28-02-2024,0.0015
1977,29-02-2024,0.0061


In [99]:
data.head()

,Date,CGS_IB_Daily
0,01-10-2018,0.0004
1,02-10-2018,-0.0005
2,03-10-2018,0.0000
3,04-10-2018,-0.0011
4,05-10-2018,-0.0001


In [100]:
data['CGS_IB_Daily'] = pd.to_numeric(data['CGS_IB_Daily'])

In [101]:
(1 + data['CGS_IB_Daily']).cumprod().iloc[-1]

1.5420112632444825

In [102]:
#Remove Saturday and Sunday in dates.

In [103]:
data1=data.copy()

In [104]:
data1['Date'] = pd.to_datetime(data1['Date'], format='%d-%m-%Y')

In [105]:
data1['Day_of_Week'] = data1['Date'].dt.strftime('%A')

In [106]:
data1.shape

(1978, 3)

In [107]:
data1.head()

,Date,CGS_IB_Daily,Day_of_Week
0,2018-10-01,0.0004,Monday
1,2018-10-02,-0.0005,Tuesday
2,2018-10-03,0.0000,Wednesday
3,2018-10-04,-0.0011,Thursday
4,2018-10-05,-0.0001,Friday


In [108]:
a=data1[data1['Day_of_Week']=='Saturday']

In [109]:
a['CGS_IB_Daily'].sum()

0.0

In [110]:
a['CGS_IB_Daily'].max()

0.0

In [111]:
a['CGS_IB_Daily'].min()

0.0

In [112]:
#All Saturdays have a 0% return so we can simply drop Saturdays

In [113]:
data2 = data1[data1['Day_of_Week'] != 'Saturday']

In [114]:
data2.shape

(1696, 3)

In [115]:
data2.head(7)

,Date,CGS_IB_Daily,Day_of_Week
0,2018-10-01,0.0004,Monday
1,2018-10-02,-0.0005,Tuesday
2,2018-10-03,0.0000,Wednesday
3,2018-10-04,-0.0011,Thursday
4,2018-10-05,-0.0001,Friday
6,2018-10-07,0.0000,Sunday
7,2018-10-08,-0.0002,Monday


In [116]:
b=data2[data2['Day_of_Week']=='Sunday']

In [117]:
b['CGS_IB_Daily'].sum()

-0.12589999999999998

In [118]:
b['CGS_IB_Daily'].max()

0.0031

In [119]:
b['CGS_IB_Daily'].min()

-0.0319

In [120]:
#Sunday has returns so we can't just drop Sunday. We will have to merge (compound) performance in Monday and then drop Sunday

In [121]:
data3 = data2.copy()

In [122]:
data3.shape

(1696, 3)

In [123]:
data3['CGS_IB_Daily_Adj']=0

for i in range (1,len(data3)):
    
    if data3['Day_of_Week'].iloc[i-1] == 'Sunday':
        
        data3['CGS_IB_Daily_Adj'].iloc[i] = ((1+data3['CGS_IB_Daily'].iloc[i-1])*(1+data3['CGS_IB_Daily'].iloc[i]))-1
        data3['CGS_IB_Daily_Adj'].iloc[i-1] = 0
    
    else:
        
        data3['CGS_IB_Daily_Adj'].iloc[i] = data3['CGS_IB_Daily'].iloc[i]

data3['CGS_IB_Daily_Adj'].iloc[0] = data3['CGS_IB_Daily'].iloc[0] 

C:\Users\Vedant Wanchoo\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [124]:
data3.shape

(1696, 4)

In [125]:
data3=data3[data3['Day_of_Week']!= 'Sunday']

In [126]:
data3.head(8)

,Date,CGS_IB_Daily,Day_of_Week,CGS_IB_Daily_Adj
0,2018-10-01,0.0004,Monday,0.0004
1,2018-10-02,-0.0005,Tuesday,-0.0005
2,2018-10-03,0.0000,Wednesday,0.0000
3,2018-10-04,-0.0011,Thursday,-0.0011
4,2018-10-05,-0.0001,Friday,-0.0001
7,2018-10-08,-0.0002,Monday,-0.0002
8,2018-10-09,-0.0002,Tuesday,-0.0002
9,2018-10-10,-0.0032,Wednesday,-0.0032


In [127]:
(1 + data3['CGS_IB_Daily_Adj']).cumprod().iloc[-1]

1.54201126324448

In [128]:
(1 + data['CGS_IB_Daily']).cumprod().iloc[-1]

1.5420112632444825

In [129]:
##
#data3.to_csv('CGS_IB_Daily_Cleaned_2023-09-26.csv')

In [130]:
#We have daily data from IB that does not include Saturdays and Sundays.

#Now we will convert daily to Monthly

In [131]:
data4=data3.copy()

In [132]:
data4.shape

(1414, 4)

In [133]:
data4.head()

,Date,CGS_IB_Daily,Day_of_Week,CGS_IB_Daily_Adj
0,2018-10-01,0.0004,Monday,0.0004
1,2018-10-02,-0.0005,Tuesday,-0.0005
2,2018-10-03,0.0000,Wednesday,0.0000
3,2018-10-04,-0.0011,Thursday,-0.0011
4,2018-10-05,-0.0001,Friday,-0.0001


In [134]:
# Set the 'Date' column as the index
data4.set_index('Date', inplace=True)

In [135]:
# Calculate daily compounded returns
data4['Compound_Return'] = (1 + data4['CGS_IB_Daily_Adj']).cumprod()

# Resample the data to monthly frequency
monthly_returns = data4['Compound_Return'].resample('M').ffill()

# Calculate the monthly compounded returns
monthly_compounded_returns = monthly_returns.pct_change().fillna(0)

# Print the resulting monthly compounded returns
#print(monthly_compounded_returns)


In [136]:
data4.head()

,CGS_IB_Daily,Day_of_Week,CGS_IB_Daily_Adj,Compound_Return
Date,,,,
2018-10-01,0.0004,Monday,0.0004,1.0004
2018-10-02,-0.0005,Tuesday,-0.0005,0.9999
2018-10-03,0.0000,Wednesday,0.0000,0.9999
2018-10-04,-0.0011,Thursday,-0.0011,0.9988
2018-10-05,-0.0001,Friday,-0.0001,0.9987


In [137]:
monthly_compounded_returns[0] = monthly_returns[0]-1

In [138]:
CGS_M=pd.DataFrame(monthly_compounded_returns)

In [139]:
CGS_M.columns=['CGS_IB_Monthly']

In [140]:
CGS_M.head()

,CGS_IB_Monthly
Date,
2018-10-31,-0.034441
2018-11-30,0.020878
2018-12-31,-0.068742
2019-01-31,0.077136
2019-02-28,0.016695


In [141]:
CGS_M.shape

(65, 1)

In [142]:
(1 + CGS_M['CGS_IB_Monthly']).cumprod().iloc[-1]

1.5420112632444787

In [143]:
(1 + data3['CGS_IB_Daily_Adj']).cumprod().iloc[-1]

1.54201126324448

In [144]:
(1 + data['CGS_IB_Daily']).cumprod().iloc[-1]

1.5420112632444825

In [145]:
##
#CGS_M.to_csv('CGS_IB_Monthly_Cleaned_2023-09-26.csv')

In [146]:
#Input official CGS monthly data from administrator

In [147]:
data_M=pd.read_csv(r'C:\Users\Vedant Wanchoo\Desktop\CGS 2020\Data Requests\CGS_Official_Monthly_2024-03-19.csv')

In [148]:
data_M.head()

,Date,CGS_Gross_Admin_Monthly,CGS_Net_Admin_Monthly
0,31-10-2018,-0.049726,-0.049726
1,30-11-2018,0.003295,0.003295
2,31-12-2018,-0.025774,-0.025774
3,31-01-2019,0.072237,0.072237
4,28-02-2019,0.014037,0.014037


In [149]:
data_M.shape

(65, 3)

In [150]:
(1 + data_M['CGS_Gross_Admin_Monthly']).cumprod().iloc[-1]

1.6659460753462085

In [151]:
(1 + data_M['CGS_Net_Admin_Monthly']).cumprod().iloc[-1]

1.6659460753462085

In [152]:
(1 + CGS_M['CGS_IB_Monthly']).cumprod().iloc[-1]

1.5420112632444787

In [153]:
(1 + data3['CGS_IB_Daily_Adj']).cumprod().iloc[-1]

1.54201126324448

In [154]:
#The official monthly numbers do not match the Interactive Brokers numbers

#We will calculate official daily numbers, using the IB daily numbers as a starting point

In [155]:
data5=data3.copy()

In [156]:
data5.shape

(1414, 4)

In [157]:
data5.head()

,Date,CGS_IB_Daily,Day_of_Week,CGS_IB_Daily_Adj
0,2018-10-01,0.0004,Monday,0.0004
1,2018-10-02,-0.0005,Tuesday,-0.0005
2,2018-10-03,0.0000,Wednesday,0.0000
3,2018-10-04,-0.0011,Thursday,-0.0011
4,2018-10-05,-0.0001,Friday,-0.0001


In [158]:
data5=data5[['Date','CGS_IB_Daily_Adj']]

In [159]:
data5['Date']= pd.to_datetime(data5['Date'])

In [160]:
data5.set_index('Date', inplace=True)

In [161]:
data5.shape

(1414, 1)

In [162]:
data5.head()

,CGS_IB_Daily_Adj
Date,
2018-10-01,0.0004
2018-10-02,-0.0005
2018-10-03,0.0000
2018-10-04,-0.0011
2018-10-05,-0.0001


In [163]:
(1 + data5['CGS_IB_Daily_Adj']).cumprod().iloc[-1]

1.54201126324448

In [164]:
(1 + data3['CGS_IB_Daily_Adj']).cumprod().iloc[-1]

1.54201126324448

In [165]:
CGS_Official_M = data_M[['Date','CGS_Net_Admin_Monthly']]

In [166]:
CGS_Official_M.shape

(65, 2)

In [167]:
CGS_Official_M['Date'] = pd.to_datetime(CGS_Official_M['Date'], format='%d-%m-%Y')

C:\Users\VEDANT~1\AppData\Local\Temp/ipykernel_13748/2468456710.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CGS_Official_M['Date'] = pd.to_datetime(CGS_Official_M['Date'], format='%d-%m-%Y')


In [168]:
CGS_Official_M.set_index('Date', inplace=True)

In [169]:
CGS_Official_M.head()

,CGS_Net_Admin_Monthly
Date,
2018-10-31,-0.049726
2018-11-30,0.003295
2018-12-31,-0.025774
2019-01-31,0.072237
2019-02-28,0.014037


In [170]:
(1 + CGS_Official_M['CGS_Net_Admin_Monthly']).cumprod().iloc[-1]

1.6659460753462085

In [171]:
(1 + data_M['CGS_Net_Admin_Monthly']).cumprod().iloc[-1]

1.6659460753462085

In [172]:
# The codes below till the next .head() have arrived out of a lot of figuring out. Have removed the .head() to save space.

In [173]:
data5['Cumulative_Return_Daily'] = (1 + data5['CGS_IB_Daily_Adj']).groupby(pd.Grouper(freq='M')).cumprod()-1

In [174]:
data5['Scaling_Factor'] = ((1+CGS_Official_M['CGS_Net_Admin_Monthly']) / (1+data5['Cumulative_Return_Daily']))
data5['Scaling_Factor'] = data5['Scaling_Factor'].fillna(method='bfill')

In [175]:
data5.reset_index(inplace=True)
data5['Month'] = data5['Date'].dt.month
data5['Year'] = data5['Date'].dt.year
data5['Days_in_Month_Year'] = data5.groupby(['Year', 'Month'])['Date'].transform('count')

In [176]:
data5['Scaling_Factor_2.0'] = (data5['Scaling_Factor'] ** (1/data5['Days_in_Month_Year']))-1

In [177]:
data5['CGS_IB_&_Official_Daily_Adj'] = data5['CGS_IB_Daily_Adj'] + data5['Scaling_Factor_2.0']

In [178]:
data5.set_index('Date', inplace=True)

In [179]:
data5.tail(40)

,CGS_IB_Daily_Adj,Cumulative_Return_Daily,Scaling_Factor,Month,Year,Days_in_Month_Year,Scaling_Factor_2.0,CGS_IB_&_Official_Daily_Adj
Date,,,,,,,,
2024-01-05,-0.0031,-0.003029,1.001173,1,2024,23,0.000051,-0.003049
2024-01-08,-0.0052,-0.008214,1.001173,1,2024,23,0.000051,-0.005149
2024-01-09,-0.0026,-0.010792,1.001173,1,2024,23,0.000051,-0.002549
2024-01-10,0.0055,-0.005352,1.001173,1,2024,23,0.000051,0.005551
2024-01-11,0.0082,0.002804,1.001173,1,2024,23,0.000051,0.008251
2024-01-12,0.0075,0.010326,1.001173,1,2024,23,0.000051,0.007551
2024-01-15,0.0011,0.011437,1.001173,1,2024,23,0.000051,0.001151
2024-01-16,0.0025,0.013965,1.001173,1,2024,23,0.000051,0.002551
2024-01-17,0.0036,0.017616,1.001173,1,2024,23,0.000051,0.003651


In [180]:
data5.tail(0)

,CGS_IB_Daily_Adj,Cumulative_Return_Daily,Scaling_Factor,Month,Year,Days_in_Month_Year,Scaling_Factor_2.0,CGS_IB_&_Official_Daily_Adj
Date,,,,,,,,


In [181]:
(1 + data3['CGS_IB_Daily_Adj']).cumprod().iloc[-1]

1.54201126324448

In [182]:
(1 + data5['CGS_IB_Daily_Adj']).cumprod().iloc[-1]

1.54201126324448

In [183]:
(1 + CGS_Official_M['CGS_Net_Admin_Monthly']).cumprod().iloc[-1]

1.6659460753462085

In [184]:
(1 + data5['CGS_IB_&_Official_Daily_Adj']).cumprod().iloc[-1]

1.6679028782321335

In [185]:
data6=data5[['CGS_IB_Daily_Adj','CGS_IB_&_Official_Daily_Adj']]

In [186]:
data6.to_csv('CGS_Daily_Output_Gross_2023-03-19.csv')

In [92]:
data6['CGS_IB_Monthly_Adj'] = (1 + data6['CGS_IB_Daily_Adj']).cumprod()
data6['CGS_IB_&_Official_Monthly_Adj'] = (1 + data6['CGS_IB_&_Official_Daily_Adj']).cumprod()

# Resample to monthly frequency and compound returns within each month
monthly_data = data6.resample('M').apply(lambda x: (x + 1).prod() - 1)

C:\Users\VEDANT~1\AppData\Local\Temp/ipykernel_1264/3485122736.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data6['CGS_IB_Monthly_Adj'] = (1 + data6['CGS_IB_Daily_Adj']).cumprod()
C:\Users\VEDANT~1\AppData\Local\Temp/ipykernel_1264/3485122736.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data6['CGS_IB_&_Official_Monthly_Adj'] = (1 + data6['CGS_IB_&_Official_Daily_Adj']).cumprod()


In [93]:
monthly_data1=monthly_data.iloc[:,0:2]

In [94]:
monthly_data1.columns=['CGS_IB_Monthly_Adj','CGS_IB_&_Official_Monthly_Adj']

In [95]:
monthly_data1.head()

,CGS_IB_Monthly_Adj,CGS_IB_&_Official_Monthly_Adj
Date,,
2018-10-31,-0.034441,-0.031005
2018-11-30,0.020878,0.013617
2018-12-31,-0.068742,-0.064937
2019-01-31,0.077136,0.072256
2019-02-28,0.016695,0.014042


In [96]:
(1 + monthly_data1['CGS_IB_Monthly_Adj']).cumprod().iloc[-1]

1.4227792232636263

In [97]:
(1 + data5['CGS_IB_Daily_Adj']).cumprod().iloc[-1]

1.4227792232636318

In [98]:
(1 + monthly_data1['CGS_IB_&_Official_Monthly_Adj']).cumprod().iloc[-1]

1.504262040220051

In [99]:
(1 + data5['CGS_IB_&_Official_Daily_Adj']).cumprod().iloc[-1]

1.504262040220049

In [100]:
monthly_data1.shape

(59, 2)

In [101]:
CGS_Official_M.shape

(59, 1)

In [102]:
CGS_Official_M.head()

,CGS_Net_Admin_Monthly
Date,
2018-10-31,-0.03101
2018-11-30,0.01361
2018-12-31,-0.06495
2019-01-31,0.07224
2019-02-28,0.01404


In [103]:
merged_df = pd.concat([CGS_Official_M, monthly_data1], axis=1)

In [104]:
merged_df.shape

(59, 3)

In [105]:
merged_df.head()

,CGS_Net_Admin_Monthly,CGS_IB_Monthly_Adj,CGS_IB_&_Official_Monthly_Adj
Date,,,
2018-10-31,-0.03101,-0.034441,-0.031005
2018-11-30,0.01361,0.020878,0.013617
2018-12-31,-0.06495,-0.068742,-0.064937
2019-01-31,0.07224,0.077136,0.072256
2019-02-28,0.01404,0.016695,0.014042


In [106]:
merged_df.to_csv('CGS_Monthly_Output_2023-09-27.csv')

In [107]:
#I eye balled the data and it matches till the second decimal point